In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

In [2]:
import random
import csv
import re
import pandas as pd

In [94]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [3]:
service = Service(ChromeDriverManager().install())

In [4]:
chrome_options = Options()
chrome_options.add_argument("--user-data-dir=C:\\Users\\HP\\AppData\\Local\\Google\\Chrome\\User Data\\Profile 2")
chrome_options.add_argument("--profile-directory=Profile 2")

In [81]:
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.get("https://www.linkedin.com/feed")

In [107]:
def get_actual_date(date):
    today = datetime.today().strftime('%Y-%m-%d')
    current_year = datetime.today().strftime('%Y')

    def get_past_date(days=0, weeks=0, months=0, years=0):
        date_format = '%Y-%m-%d'
        dtObj = datetime.strptime(today, date_format)
        past_date = dtObj - relativedelta(days=days, weeks=weeks, months=months, years=years)
        past_date_str = past_date.strftime(date_format)
        return past_date_str

    # Remove any extra text after "•"
    date = re.sub(r"•.*", "", date).strip()

    # Extract the number and the time unit (e.g., '5d', '2w', '3m', etc.)
    match = re.match(r"(\d+)([a-zA-Z]+)", date)
    
    if match:
        number = int(match.group(1))  # The number (e.g., 5 from '5d')
        unit = match.group(2).lower()  # The unit (e.g., 'd' from '5d')


        if unit == 'm' or unit == 'h':
            past_date = today
        if unit == 'd':  # Days
            past_date = get_past_date(days=number)
        elif unit == 'w':  # Weeks
            past_date = get_past_date(weeks=number)
        elif unit == 'mo':  # Months
            past_date = get_past_date(months=number)
        elif unit == 'y':  # Years
            past_date = get_past_date(years=number)
        else:
            past_date = today  # Default to today if the unit is unknown
    else:
        # If no match, try parsing a specific date (format: MM-DD or MM-DD-YYYY)
        split_date = date.split("-")
        if len(split_date) == 2:  # MM-DD
            past_month = split_date[0]
            past_day = split_date[1]
            if len(past_month) < 2:
                past_month = "0" + past_month
            if len(past_day) < 2:
                past_day = "0" + past_day
            past_date = f"{current_year}-{past_month}-{past_day}"
        elif len(split_date) == 3:  # MM-DD-YYYY
            past_month = split_date[0]
            past_day = split_date[1]
            past_year = split_date[2]
            if len(past_month) < 2:
                past_month = "0" + past_month
            if len(past_day) < 2:
                past_day = "0" + past_day
            past_date = f"{past_year}-{past_month}-{past_day}"
        else:
            past_date = today  # Default to today if the format is unrecognized

    return past_date

In [80]:
post_elements = driver.find_elements(By.CLASS_NAME, "fie-impression-container")
for post in post_elements:
    # social_reach_div = post.find_element(By.CSS_SELECTOR, "span.social-details-social-counts__reactions-count")
    # print(social_reach_div.text.strip())


    media_type = "text"
    media_links = []
    media_count = 0

    # Check for Video
    if post.find_elements(By.CSS_SELECTOR, ".update-components-linkedin-video"):
        media_type = "video"
        try:
            video_element = post.find_element(By.CSS_SELECTOR, "video")
            video_src = video_element.get_attribute("src")
            media_links = [video_src]  # Store as list for consistency
            media_count = 1
        except:
            media_links = []
            media_count = 0

    # Check for Image(s)
    elif post.find_elements(By.CSS_SELECTOR, ".update-components-image"):
        media_type = "image"
        try:
            image_elements = post.find_elements(By.CSS_SELECTOR, ".update-components-image img.ivm-view-attr__img--centered")
            media_links = [img.get_attribute("src") for img in image_elements]
            media_count = len(media_links)
        except:
            media_links = []
            media_count = 0

    elif post.find_elements(By.CSS_SELECTOR, ".feed-shared-article__container") or \
        post.find_elements(By.CSS_SELECTOR, ".feed-shared-link__container") or \
        post.find_elements(By.CSS_SELECTOR, ".update-components-document__container"):

        media_type = "article"
        try:
            iframe_elements = post.find_elements(By.CSS_SELECTOR, "iframe")
            media_links = []

            for iframe in iframe_elements:
                iframe_src = iframe.get_attribute("src")

                # Check if the iframe src contains the specific document link
                if iframe_src and "feedshare-document-images" in iframe_src:
                    media_links.append(iframe_src)  # Append the iframe src (which is the image source)

            media_count = len(media_links)


        except Exception as e:
            print(f"Error extracting images from article: {e}")
            media_links = []
            media_count = 0

    # Fallback: Text-only post
    else:
        media_type = "text"
        media_links = []
        media_count = 0


    # has_article = bool(post.find_elements(By.CSS_SELECTOR, ".feed-shared-article__container"))
    # has_link = bool(post.find_elements(By.CSS_SELECTOR, ".feed-shared-link__container"))
    # has_document = bool(post.find_elements(By.CSS_SELECTOR, ".update-components-document__container"))

    # print(f"Article: {has_article}, Link: {has_link}, Document: {has_document}")

    if(media_type == "article"):
        print(media_type)
        print(media_count)
        print(media_links)
        print("\n")

article
0
[]




In [ ]:
main_df = []

count = 5
while (count>0):

    post_elements = driver.find_elements(By.CLASS_NAME, "fie-impression-container")

    for post in post_elements:
        try:
            header_div = post.find_element(By.CSS_SELECTOR, "div.update-components-actor__container")
            
            try:
                header_title = header_div.find_element(By.CSS_SELECTOR, "span.update-components-actor__title span[aria-hidden='true']").text
            except:
                header_title = None

            try:
                header_description = header_div.find_element(By.CSS_SELECTOR, "span.update-components-actor__description span[aria-hidden='true']").text
            except:
                header_description = None

            try:        
                header_sub_description = header_div.find_element(By.CSS_SELECTOR, "span.update-components-actor__sub-description span[aria-hidden='true']").text
            except:
                header_sub_description = None


        except Exception as e:
            header_title = None


        try:
            content_div = post.find_element(By.CSS_SELECTOR, "div.update-components-text")

            try:
                content_text = content_div.text
            except:
                content_text = None
        
        except:
            content_div = None


        try:
            social_reach_div = post.find_element(By.CSS_SELECTOR, ".social-details-social-counts")
            
            try:
                reaction_div = social_reach_div.find_element(By.CSS_SELECTOR, "span.social-details-social-counts__reactions-count")
                reaction_count = reaction_div.text.strip()
            except:
                reaction_count = 0

            try:
                comment_div = social_reach_div.find_element(By.CSS_SELECTOR, "button[aria-label*='comment'] span[aria-hidden='true']")
                comment_count = comment_div.text.strip()
            except:
                comment_count = 0

            try:
                repost_div = social_reach_div.find_element(By.CSS_SELECTOR, "button[aria-label*='reposts'] span[aria-hidden='true']")
                repost_count = comment_div.text.strip()
            except:
                repost_count = 0

            media_type = "text"
            media_links = []
            media_count = 0

            # Check for Video
            if post.find_elements(By.CSS_SELECTOR, ".update-components-linkedin-video"):
                media_type = "video"
                try:
                    video_element = post.find_element(By.CSS_SELECTOR, "video")
                    video_src = video_element.get_attribute("src")
                    media_links = [video_src]  # Store as list for consistency
                    media_count = 1
                except:
                    media_links = []
                    media_count = 0

            # Check for Image(s)
            elif post.find_elements(By.CSS_SELECTOR, ".update-components-image"):
                media_type = "image"
                try:
                    image_elements = post.find_elements(By.CSS_SELECTOR, ".update-components-image img.ivm-view-attr__img--centered")
                    media_links = [img.get_attribute("src") for img in image_elements]
                    media_count = len(media_links)
                except:
                    media_links = []
                    media_count = 0

            elif post.find_elements(By.CSS_SELECTOR, ".feed-shared-article__container") or \
                post.find_elements(By.CSS_SELECTOR, ".feed-shared-link__container") or \
                post.find_elements(By.CSS_SELECTOR, ".update-components-document__container"):

                media_type = "article"
                try:
                    iframe_elements = post.find_elements(By.CSS_SELECTOR, "iframe")
                    media_links = []

                    for iframe in iframe_elements:
                        iframe_src = iframe.get_attribute("src")

                        # Check if the iframe src contains the specific document link
                        if iframe_src and "feedshare-document-images" in iframe_src:
                            media_links.append(iframe_src)  # Append the iframe src (which is the image source)

                    media_count = len(media_links)

                except Exception as e:
                    print(f"Error extracting images from article: {e}")
                    media_links = []
                    media_count = 0

            # Fallback: Text-only post
            else:
                media_type = "text"
                media_links = []
                media_count = 0
        
        except:
            social_reach_div = None


        main_df.append({
            "name": header_title,
            "desc": header_description,
            "timestamp": header_sub_description,
            "content": content_text,
            'reaction_count' : reaction_count,
            "comments_count" : comment_count,
            "repost_count" : repost_count,
            "Media Type": media_type,
            "Media Link": media_links,
            "Media Count": media_count
        })


    driver.execute_script("window.scrollBy(0, 500);")
    time.sleep(10)
    count = count - 1

In [83]:
main_df = pd.DataFrame(main_df)

In [84]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   name            18 non-null     object
 1   desc            38 non-null     object
 2   timestamp       38 non-null     object
 3   content         38 non-null     object
 4   reaction_count  38 non-null     object
 5   comments_count  38 non-null     object
 6   repost_count    38 non-null     object
 7   Media Type      38 non-null     object
 8   Media Link      38 non-null     object
 9   Media Count     38 non-null     int64 
dtypes: int64(1), object(9)
memory usage: 3.1+ KB


In [87]:
main_df.head(15)

,name,desc,timestamp,content,reaction_count,comments_count,repost_count,Media Type,Media Link,Media Count
0,Palak Sharma,SDE-1 @Flipkart | GSoC'22 @Moja Global | Winne...,23m • \n,🚨 🔥 5 TOP Internships for Tech Enthusiasts!\n\...,5,0,0,image,[https://media.licdn.com/dms/image/v2/D5622AQG...,1
1,aman Kour,Empowering Digital | LinkedIn influencer. 🚀 Ht...,5d • Edited • \n,ChatGPT Can Get You Hired Faster Than Any Recr...,193,193,193,article,[],0
2,None,Empowering Digital | LinkedIn influencer. 🚀 Ht...,5d • Edited • \n,ChatGPT Can Get You Hired Faster Than Any Recr...,193,193,193,article,[],0
3,None,Empowering Digital | LinkedIn influencer. 🚀 Ht...,5d • Edited • \n,ChatGPT Can Get You Hired Faster Than Any Recr...,193,193,193,article,[],0
4,None,Empowering Digital | LinkedIn influencer. 🚀 Ht...,5d • Edited • \n,ChatGPT Can Get You Hired Faster Than Any Recr...,193,193,193,article,[],0
5,None,Empowering Digital | LinkedIn influencer. 🚀 Ht...,5d • Edited • \n,ChatGPT Can Get You Hired Faster Than Any Recr...,193,193,193,article,[],0
6,Palak Sharma,SDE-1 @Flipkart | GSoC'22 @Moja Global | Winne...,23m • \n,🚨 🔥 5 TOP Internships for Tech Enthusiasts!\n\...,5,0,0,image,[https://media.licdn.com/dms/image/v2/D5622AQG...,1
7,aman Kour,Empowering Digital | LinkedIn influencer. 🚀 Ht...,5d • Edited • \n,ChatGPT Can Get You Hired Faster Than Any Recr...,193,193,193,article,[],0
8,None,Empowering Digital | LinkedIn influencer. 🚀 Ht...,5d • Edited • \n,ChatGPT Can Get You Hired Faster Than Any Recr...,193,193,193,article,[],0
9,None,Empowering Digital | LinkedIn influencer. 🚀 Ht...,5d • Edited • \n,ChatGPT Can Get You Hired Faster Than Any Recr...,193,193,193,article,[],0


In [88]:
main_df = main_df.dropna(subset=["name"])

In [89]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18 entries, 0 to 37
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   name            18 non-null     object
 1   desc            18 non-null     object
 2   timestamp       18 non-null     object
 3   content         18 non-null     object
 4   reaction_count  18 non-null     object
 5   comments_count  18 non-null     object
 6   repost_count    18 non-null     object
 7   Media Type      18 non-null     object
 8   Media Link      18 non-null     object
 9   Media Count     18 non-null     int64 
dtypes: int64(1), object(9)
memory usage: 1.5+ KB


In [90]:
main_df.head(10)

,name,desc,timestamp,content,reaction_count,comments_count,repost_count,Media Type,Media Link,Media Count
0,Palak Sharma,SDE-1 @Flipkart | GSoC'22 @Moja Global | Winne...,23m • \n,🚨 🔥 5 TOP Internships for Tech Enthusiasts!\n\...,5,0,0,image,[https://media.licdn.com/dms/image/v2/D5622AQG...,1
1,aman Kour,Empowering Digital | LinkedIn influencer. 🚀 Ht...,5d • Edited • \n,ChatGPT Can Get You Hired Faster Than Any Recr...,193,193,193,article,[],0
6,Palak Sharma,SDE-1 @Flipkart | GSoC'22 @Moja Global | Winne...,23m • \n,🚨 🔥 5 TOP Internships for Tech Enthusiasts!\n\...,5,0,0,image,[https://media.licdn.com/dms/image/v2/D5622AQG...,1
7,aman Kour,Empowering Digital | LinkedIn influencer. 🚀 Ht...,5d • Edited • \n,ChatGPT Can Get You Hired Faster Than Any Recr...,193,193,193,article,[],0
12,Palak Sharma,SDE-1 @Flipkart | GSoC'22 @Moja Global | Winne...,23m • \n,🚨 🔥 5 TOP Internships for Tech Enthusiasts!\n\...,5,0,0,image,[https://media.licdn.com/dms/image/v2/D5622AQG...,1
13,aman Kour,Empowering Digital | LinkedIn influencer. 🚀 Ht...,5d • Edited • \n,ChatGPT Can Get You Hired Faster Than Any Recr...,193,193,193,article,[],0
18,Palak Sharma,SDE-1 @Flipkart | GSoC'22 @Moja Global | Winne...,23m • \n,🚨 🔥 5 TOP Internships for Tech Enthusiasts!\n\...,5,0,0,image,[https://media.licdn.com/dms/image/v2/D5622AQG...,1
19,aman Kour,Empowering Digital | LinkedIn influencer. 🚀 Ht...,5d • Edited • \n,ChatGPT Can Get You Hired Faster Than Any Recr...,193,193,193,article,[],0
24,SciSpectra,"1,917 followers",4d • \n,"ISI, the Indian Statistical Institute...\n\nan...",380,380,380,image,[https://media.licdn.com/dms/image/v2/D5622AQH...,1
25,Vagas Empregos,"4,489,128 followers",23m • \n,Vagas no site: https://lnkd.in/dd3jFZhT,6,0,0,image,[https://media.licdn.com/dms/image/v2/D4D22AQG...,1


In [91]:
main_df.drop_duplicates(subset=["content"], inplace=True)

In [92]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6 entries, 0 to 27
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   name            6 non-null      object
 1   desc            6 non-null      object
 2   timestamp       6 non-null      object
 3   content         6 non-null      object
 4   reaction_count  6 non-null      object
 5   comments_count  6 non-null      object
 6   repost_count    6 non-null      object
 7   Media Type      6 non-null      object
 8   Media Link      6 non-null      object
 9   Media Count     6 non-null      int64 
dtypes: int64(1), object(9)
memory usage: 528.0+ bytes


In [93]:
main_df.head(22)

,name,desc,timestamp,content,reaction_count,comments_count,repost_count,Media Type,Media Link,Media Count
0,Palak Sharma,SDE-1 @Flipkart | GSoC'22 @Moja Global | Winne...,23m • \n,🚨 🔥 5 TOP Internships for Tech Enthusiasts!\n\...,5,0,0,image,[https://media.licdn.com/dms/image/v2/D5622AQG...,1
1,aman Kour,Empowering Digital | LinkedIn influencer. 🚀 Ht...,5d • Edited • \n,ChatGPT Can Get You Hired Faster Than Any Recr...,193,193,193,article,[],0
24,SciSpectra,"1,917 followers",4d • \n,"ISI, the Indian Statistical Institute...\n\nan...",380,380,380,image,[https://media.licdn.com/dms/image/v2/D5622AQH...,1
25,Vagas Empregos,"4,489,128 followers",23m • \n,Vagas no site: https://lnkd.in/dd3jFZhT,6,0,0,image,[https://media.licdn.com/dms/image/v2/D4D22AQG...,1
26,Eli Cohen,"Founder, OSMOS® | Forbes 30 Under 30 | Buildin...",4d • \n,Agencies are scamming founders out of millions...,279,279,279,text,[],0
27,Palak Sharma,SDE-1 @Flipkart | GSoC'22 @Moja Global | Winne...,6m • \n,"🚀 𝗙𝗿𝗲𝘀𝗵 𝗧𝗲𝗰𝗵 𝗝𝗼𝗯𝘀 𝗔𝗹𝗲𝗿𝘁!!\n👇 𝗦𝗮𝘃𝗲 𝘁𝗵𝗶𝘀, 𝗮𝗽𝗽𝗹𝘆 ...",279,279,279,text,[],0


In [108]:
main_df['converted_date'] = main_df['timestamp'].apply(lambda x: get_actual_date(x))

In [109]:
main_df.head()

,name,desc,timestamp,content,reaction_count,comments_count,repost_count,Media Type,Media Link,Media Count,converted_date
0,Palak Sharma,SDE-1 @Flipkart | GSoC'22 @Moja Global | Winne...,23m • \n,🚨 🔥 5 TOP Internships for Tech Enthusiasts!\n\...,5,0,0,image,[https://media.licdn.com/dms/image/v2/D5622AQG...,1,2025-07-13
1,aman Kour,Empowering Digital | LinkedIn influencer. 🚀 Ht...,5d • Edited • \n,ChatGPT Can Get You Hired Faster Than Any Recr...,193,193,193,article,[],0,2025-07-08
24,SciSpectra,"1,917 followers",4d • \n,"ISI, the Indian Statistical Institute...\n\nan...",380,380,380,image,[https://media.licdn.com/dms/image/v2/D5622AQH...,1,2025-07-09
25,Vagas Empregos,"4,489,128 followers",23m • \n,Vagas no site: https://lnkd.in/dd3jFZhT,6,0,0,image,[https://media.licdn.com/dms/image/v2/D4D22AQG...,1,2025-07-13
26,Eli Cohen,"Founder, OSMOS® | Forbes 30 Under 30 | Buildin...",4d • \n,Agencies are scamming founders out of millions...,279,279,279,text,[],0,2025-07-09


In [98]:
mask = ~main_df['desc'].str.contains('followers', case=False, na=False)
personal_df = main_df[mask]

In [101]:
personal_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6 entries, 0 to 22
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   name            5 non-null      object
 1   desc            6 non-null      object
 2   timestamp       6 non-null      object
 3   content         6 non-null      object
 4   reaction_count  6 non-null      object
 5   comments_count  6 non-null      object
 6   repost_count    6 non-null      object
dtypes: object(7)
memory usage: 384.0+ bytes


In [68]:
def scrape_posts(driver):
    posts = []
    
    # Find all posts on the page
    post_elements = driver.find_elements(By.CLASS_NAME, "fie-impression-container")
    
    for post in post_elements:
        try:
            # Extract post text
            post_text = post.text
        except Exception as e:
            post_text = "Text not found"
        
        # try:
        #     post_time = post.find_element(By.XPATH, ".//time").get_attribute("datetime")
        # except Exception as e:
        #     post_time = "Timestamp not found"
        
        posts.append(post_text)
    
    return posts

In [69]:
def save_posts_to_csv(posts, filename="linkedin_posts.csv"):
    with open(filename, 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)

        if csvfile.tell() == 0:  # Check if file is empty
            writer.writerow(["Post Text"])

        for post in posts:
            writer.writerow([post])

    print(f"Posts appended to {filename}")

In [72]:
scroll_page(driver)

posts = scrape_posts(driver)

# Save posts to CSV
save_posts_to_csv(posts)

KeyboardInterrupt: 

In [ ]:
# time.sleep(5)

# username = driver.find_element(By.ID, "username")
# password = driver.find_element(By.ID, "password")

In [ ]:
# username_value = "vivektripathi339@gmail.com"
# password_value = "newPassCode"

In [ ]:
# time.sleep(5)
# username.send_keys(username_value)
# time.sleep(5)
# password.send_keys(password_value)

In [ ]:
# time.sleep(3)
# driver.find_element(By.XPATH, "//button[@type='submit']").click()

In [58]:
time.sleep(3)

# Scrape post text
posts = driver.find_elements(By.CLASS_NAME, "fie-impression-container")
post_data = []

In [59]:
print(posts)

[<selenium.webdriver.remote.webelement.WebElement (session="f95066eb28ee72ecf99b8100fb8f361a", element="f.45D7E4B6BB92049C919C61E781BC0305.d.01D64FE187F851E11F5D03BD888D8F08.e.974")>, <selenium.webdriver.remote.webelement.WebElement (session="f95066eb28ee72ecf99b8100fb8f361a", element="f.45D7E4B6BB92049C919C61E781BC0305.d.01D64FE187F851E11F5D03BD888D8F08.e.1343")>, <selenium.webdriver.remote.webelement.WebElement (session="f95066eb28ee72ecf99b8100fb8f361a", element="f.45D7E4B6BB92049C919C61E781BC0305.d.01D64FE187F851E11F5D03BD888D8F08.e.1355")>, <selenium.webdriver.remote.webelement.WebElement (session="f95066eb28ee72ecf99b8100fb8f361a", element="f.45D7E4B6BB92049C919C61E781BC0305.d.01D64FE187F851E11F5D03BD888D8F08.e.1390")>, <selenium.webdriver.remote.webelement.WebElement (session="f95066eb28ee72ecf99b8100fb8f361a", element="f.45D7E4B6BB92049C919C61E781BC0305.d.01D64FE187F851E11F5D03BD888D8F08.e.1425")>, <selenium.webdriver.remote.webelement.WebElement (session="f95066eb28ee72ecf99b8

In [ ]:
for post in posts:
    post_text_main = post.text
    for pText in post_text_main:
        if(pText ==)
    post_data.append(post_text)

In [61]:
df = pd.DataFrame(post_data, columns=["Post Text"])

In [62]:
df.head()

,Post Text
0,JavaScript Notes reposted this\nLeonard Rodman...
1,Recommended for you\nGeeksforGeeks\n1 DSA prob...
2,GeeksforGeeks\n1 DSA problem a day- Keeps jobl...
3,Flipkart\nGet in touch with our experts for su...
4,"UN Jobs and NGO Jobs\n""Where Your Career Meets..."


In [63]:
df.to_csv("check.csv")

In [64]:
import pandas as pd
import re

def extract_post_info(text):
    lines = text.split('\n')
    
    # Extract name and title
    name = lines[1].split(',')[0].strip() if len(lines) > 1 else None
    title = ' | '.join(lines[2].split('|')[1:]) if len(lines) > 2 else None
    
    # Clean text
    cleaned_text = '\n'.join(lines[:])
    
    # Likes, comments, shares
    likes = re.search(r'(\d+)(?=\s*like)', text, re.IGNORECASE)
    comments = re.search(r'(\d+)(?=\s*comment)', text, re.IGNORECASE)
    shares = re.search(r'(\d+)(?=\s*repost|share)', text, re.IGNORECASE)

    # Posted time
    posted_time = re.search(r'(\d+\s*[a-zA-Z]+)\s*•', text)

    return {
        'full_text': cleaned_text,
        'author_name': name,
        'author_title': title,
        'likes': int(likes.group(1)) if likes else 0,
        'comments': int(comments.group(1)) if comments else 0,
        'shares': int(shares.group(1)) if shares else 0,
        'posted_time': posted_time.group(1) if posted_time else None,
        'post_id': None,  # Not available
        'post_link': None  # Not available
    }

df = pd.read_csv('check.csv')

results = []
for idx, row in df.iterrows():
    results.append(extract_post_info(row['Post Text']))

output_df = pd.DataFrame(results)
output_df.to_csv('linkedin_posts_parsed.csv', index=False)

In [ ]:
<video id="ember720_html5_api" class="vjs-tech" tabindex="-1" role="application" preload="metadata" muted="muted" poster="https://media.licdn.com/dms/image/v2/D4D05AQEzLw1VCwnBeg/videocover-high/B4DZfzknQNGgCU-/0/1752138168297?e=1752966000&amp;v=beta&amp;t=-07dUHvmPS-2Ig5_GDBckVqomxO7rsip1p5SrCPhpQo" src="blob:https://www.linkedin.com/2577437d-a5ab-46a7-b8c9-fde5901a6e59" autoplay="autoplay"></video>